# 

In [1]:
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
import os

# Set environment variables
os.environ["BASE_URL"] = "http://localhost:8000/v1"
os.environ["OPENAI_API_KEY"] = "abc-123"

# Create OpenAI-compatible provider
provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

# Reference the model using the name from --served-model-name
agent_model = OpenAIModel("llama3-70b", provider=provider)


In [2]:
! curl http://localhost:8000/v1/models -H "Authorization: Bearer abc-123"


{"object":"list","data":[{"id":"llama3-70b","object":"model","created":1752973487,"owned_by":"vllm","root":"/home/user/Models/meta-llama/Llama-3.3-70B-Instruct","parent":null,"max_model_len":8192,"permission":[{"id":"modelperm-e890d5a149684133aa450615c925c7b3","object":"model_permission","created":1752973487,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [3]:
!pip install -q pydantic_ai openai     

In [4]:
from pydantic_ai import Agent

agent = Agent(
    model=agent_model
)

In [5]:
import asyncio
from pydantic_ai.mcp import MCPServerStdio
async def run_async(prompt: str) -> str:
    async with agent.run_mcp_servers():
        result = await agent.run(prompt)
        return result.output


In [6]:
await run_async("What is the capital of France?")

'The capital of France is Paris.'

In [7]:
from datetime import datetime, timedelta
from typing import Dict
from datetime import datetime
from pydantic_ai import Tool  

@Tool
def calculate_meeting_times(ref_datetime_str: str, target_day: str) -> Dict[str, str]:
    """
    Given a reference starting datetime string, Calculates the start and end datetime
    for a meeting on the next occurrence of a given day of the week.

    Args:
        ref_datetime_str: The starting datetime as a string in the format
                          "YYYY-MM-DD HH:MM:SS".
        target_day: The desired day of the week (e.g., "monday").

    Returns:
        A dictionary with "START_TIME" and "END_TIME" strings.
    
    Raises:
        ValueError: If an invalid target_day or datetime format is provided.
    """
    
    try:
        # The format code for "YYYY-MM-DD HH:MM:SS" is "%Y-%m-%d %H:%M:%S"
        ref_datetime = datetime.strptime(ref_datetime_str, "%Y-%m-%d %H:%M:%S")
    except ValueError:
        raise ValueError("Invalid datetime format. Please use 'YYYY-MM-DD HH:MM:SS'.")

    
    days_of_week = {
        "monday": 0, "tuesday": 1, "wednesday": 2, "thursday": 3,
        "friday": 4, "saturday": 5, "sunday": 6
    }
    
    target_day_lower = target_day.lower()
    if target_day_lower not in days_of_week:
        raise ValueError(f"Invalid target_day: '{target_day}'.")

    ref_day_num = ref_datetime.weekday()
    target_day_num = days_of_week[target_day_lower]

    days_to_add = (target_day_num - ref_day_num) % 7
    if days_to_add == 0:
        days_to_add = 7
    
    meeting_date = (ref_datetime + timedelta(days=days_to_add)).date()

    
    start_time_str = f"{meeting_date} 09:00:00"
    end_time_str = f"{meeting_date} 10:00:00"

    return {
        "START_TIME": start_time_str,
        "END_TIME": end_time_str
    }

In [8]:
await run_async("What is the capital of France?")

'The capital of France is Paris.'

In [14]:
input_dict = {
        "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
        "Datetime": "19-07-2025T12:34:55",
        "Location": "IISc Bangalore",
        "From": "userone.amd@gmail.com",
        "Attendees": [
            {
                "email": "usertwo.amd@gmail.com"
            },
            {
                "email": "userthree.amd@gmail.com"
            }
        ],
        "Subject": "Objectives for next financial year",
        "EmailContent": "hey guys can we hold a meeting on Tuesday at 3pm for like 30 minnutes?"
    }

In [69]:
creator = input_dict["From"]

try:
    reference_datetime = datetime.strptime(input_dict["Datetime"], "%d-%m-%YT%H:%M:%S")
except ValueError:
    reference_datetime = datetime.fromisoformat(input_dict["Datetime"].replace('T', ' '))

# Parse attendees (no LLM needed)
attendees = []
for attendee in input_dict["Attendees"]:
    attendees.append(attendee["email"])

attendees.append(input_dict["From"])

reference_day = reference_datetime.strftime("%A")
reference_date_str = reference_datetime.strftime("%Y-%m-%d")

agent = Agent(
    model=agent_model,
    tools = [calculate_meeting_times],
        system_prompt =( """You are an expert at parsing meeting requests and calculating dates relative to a reference point.

CRITICAL: You MUST respond ONLY in the exact format specified below. Do not include any explanations, notes, or additional text.

YOU HAVE ACCESS TO THE FOLLOWING TOOLS:
"calculate_meeting_times(ref_datetime_str: str, target_day: str)\n"
Use this tool to get start and end datetime for a meeting on the next occurrence of a given day of the week


DATE CALCULATION LOGIC:
The reference datetime is your anchor point. From this point, calculate the target date by counting forward:

1. RELATIVE TERMS:
    - "today" = reference date (same day)
    - "tomorrow" = reference date + 1 day
    - "next [day]" = first occurrence of that day after reference date
    - "next working day" = next Monday-Friday after reference date

2. WEEKDAY NAMES (Monday, Tuesday, etc.):
    - Only use the calculate_meeting_times(...) tool when the user mentions a weekday by name (like "Monday", "Thursday", etc.). If the user gives a specific date, relative phrase like "tomorrow", or a full datetime, handle it without using the tool.
3. IF Date of meeting mentioned:
    - Then schedule it on that particular day.

TIME PARSING RULES:
- Extract specific times: "3 pm" -> 15:00, "9:30 AM" -> 09:30
- Extract duration: "30 minutes" -> 0.5 hours, "2 hours" -> 2.0 hours
- Default time if none specified: 09:00 AM
- Default duration if none specified: 1 hour
- Make sure 
- Time windows: "morning" = 09:00-12:00, "afternoon" = 13:00-17:00, "evening" = 17:00-20:00
- Dates: 2nd -> 2nd day of the month etc

MANDATORY OUTPUT FORMAT (copy exactly, replace values):
START_TIME: 
END_TIME: 
SUMMARY:

You must provide ALL 3 lines in exactly this format with valid datetime values in YYYY-MM-DD HH:MM:SS format.""" 
                         # This closing part is crucial
)
)

user_prompt = f"""REFERENCE POINT:
Datetime: {reference_datetime.strftime('%Y-%m-%d %H:%M:%S')}
Day: {reference_day}

MEETING REQUEST TO PARSE:
Subject: {input_dict["Subject"]}
Email: {input_dict["EmailContent"]}
"""



In [70]:
output = await run_async(user_prompt)
print(output)  

🔧 TOOL ACCESSED: YES
🔧 TOOL INPUT - Reference datetime: 2025-07-19 12:34:55
🔧 TOOL INPUT - Target day: tuesday
🔧 TOOL OUTPUT: {'START_TIME': '2025-07-22 09:00:00', 'END_TIME': '2025-07-22 10:00:00'}
🔧 TOOL CALCULATION: Saturday 2025-07-19 + 3 days = Tuesday 2025-07-22
START_TIME: 2025-07-23 16:00:00
END_TIME: 2025-07-23 17:00:00
SUMMARY: queen visiting company


In [60]:
@Tool
def calculate_meeting_times(ref_datetime_str: str, target_day: str) -> Dict[str, str]:
    """
    Given a reference starting datetime string, calculates the start and end datetime
    for a meeting on the next occurrence of a given day of the week.
    
    Args:
        ref_datetime_str: The starting datetime as a string in the format
                          "YYYY-MM-DD HH:MM:SS".
        target_day: The desired day of the week (e.g., "monday", "tuesday", etc.).
    
    Returns:
        A dictionary with "START_TIME" and "END_TIME" strings in YYYY-MM-DD HH:MM:SS format.
    
    Raises:
        ValueError: If an invalid target_day or datetime format is provided.
    """
    
    # DEBUG: Print when tool is accessed
    print("🔧 TOOL ACCESSED: YES")
    print(f"🔧 TOOL INPUT - Reference datetime: {ref_datetime_str}")
    print(f"🔧 TOOL INPUT - Target day: {target_day}")
    
    try:
        ref_datetime = datetime.strptime(ref_datetime_str, "%Y-%m-%d %H:%M:%S")
    except ValueError:
        raise ValueError("Invalid datetime format. Please use 'YYYY-MM-DD HH:MM:SS'.")
    
    days_of_week = {
        "monday": 0, "tuesday": 1, "wednesday": 2, "thursday": 3,
        "friday": 4, "saturday": 5, "sunday": 6
    }
    
    target_day_lower = target_day.lower()
    if target_day_lower not in days_of_week:
        raise ValueError(f"Invalid target_day: '{target_day}'.")
    
    ref_day_num = ref_datetime.weekday()
    target_day_num = days_of_week[target_day_lower]
    days_to_add = (target_day_num - ref_day_num) % 7
    if days_to_add == 0:  # If it's the same day, get next week
        days_to_add = 7
    
    meeting_date = (ref_datetime + timedelta(days=days_to_add)).date()
    
    # Return with default 9 AM start, 1 hour duration
    start_time_str = f"{meeting_date} 09:00:00"
    end_time_str = f"{meeting_date} 10:00:00"
    
    result = {
        "START_TIME": start_time_str,
        "END_TIME": end_time_str
    }
    
    # DEBUG: Print what tool returns
    print(f"🔧 TOOL OUTPUT: {result}")
    print(f"🔧 TOOL CALCULATION: {ref_datetime.strftime('%A %Y-%m-%d')} + {days_to_add} days = {meeting_date.strftime('%A %Y-%m-%d')}")
    
    return result

In [75]:
async def process_meeting_request(input_dict, agent_model):
    """Process meeting request and return parsed meeting details"""
    
    # Parse creator and attendees
    creator = input_dict["From"]
    
    # Parse reference datetime
    try:
        reference_datetime = datetime.strptime(input_dict["Datetime"], "%d-%m-%YT%H:%M:%S")
    except ValueError:
        reference_datetime = datetime.fromisoformat(input_dict["Datetime"].replace('T', ' '))
    
    # Parse attendees
    attendees = []
    for attendee in input_dict["Attendees"]:
        attendees.append(attendee["email"])
    attendees.append(input_dict["From"])
    
    # Create agent with the tool
    agent = Agent(
        model=agent_model,
        tools=[calculate_meeting_times],  # Pass the tool here
        system_prompt = f"""You are an expert at parsing meeting requests and calculating dates relative to a reference point.

REFERENCE INFORMATION:
- Current datetime: {reference_datetime.strftime('%Y-%m-%d %H:%M:%S')}
- Current day: {reference_datetime.strftime('%A')}

CRITICAL TOOL USAGE RULES:
1. When the user mentions a weekday by NAME (Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday):
   - YOU MUST use the calculate_meeting_times(ref_datetime_str, target_day) tool
   - The tool will return the CORRECT DATE - you MUST use this exact date
   - NEVER calculate the date yourself when a weekday name is mentioned
   - Only adjust the TIME portion based on user input (e.g., "4 pm" = 16:00:00)

2. DO NOT use the tool for: "tomorrow", "today", "next week", specific dates like "17th", "January 15th"

STEP-BY-STEP PROCESS FOR WEEKDAY NAMES:
1. Call the tool with the weekday name
2. Extract the DATE from tool's START_TIME (ignore the 09:00:00 time part)
3. Parse the user's requested time (e.g., "4 pm" = 16:00:00)
4. Combine: tool's DATE + user's TIME = final START_TIME
5. Add 1 hour for END_TIME (unless duration specified)

EXAMPLE:
- Tool returns: START_TIME: "2025-07-22 09:00:00" 
- User wants: "Tuesday at 4 pm"
- Your output: START_TIME: "2025-07-22 16:00:00", END_TIME: "2025-07-22 17:00:00"

DATE CALCULATION LOGIC FOR NON-WEEKDAYS:
- "today" = reference date
- "tomorrow" = reference date + 1 day  
- "next [day]" = first occurrence of that day after reference date
- Specific dates: Calculate directly (e.g., "17th" = 17th of current/next month)

TIME PARSING:
- Extract times: "3 pm" -> 15:00, "9:30 AM" -> 09:30
- Default time if none: 09:00
- Default duration: 1 hour
- Time windows: "morning" = 09:00, "afternoon" = 14:00, "evening" = 18:00

PRIORITY LEVELS:
1 = FIXED (unchangeable events like appointments, deadlines, client meetings)
2 = URGENT (time-sensitive but can be slightly adjusted if needed)
3 = IMPORTANT (significant events but with some scheduling flexibility)
4 = FLEXIBLE (can be easily rescheduled or moved)

MANDATORY OUTPUT FORMAT:
START_TIME: YYYY-MM-DD HH:MM:SS
END_TIME: YYYY-MM-DD HH:MM:SS
SUMMARY: [Brief meeting summary]
PRIORITY: [1/2/3/4]

VERIFICATION STEP:
Before providing your final answer, verify that if you used the tool, your START_TIME uses the same date as the tool's output if it was accessed"""
    )
    
    # Create user prompt
    user_prompt = f"""MEETING REQUEST TO PARSE:
Subject: {input_dict["Subject"]}
Email Content: {input_dict["EmailContent"]}

Please parse this meeting request and provide the start time, end time, and summary."""
    
    # Run the agent
    result = await agent.run(user_prompt)
    return result.output

In [72]:
input_dict = {
        "Request_id": "6118b54f-907b-4451-8d48-dd13d76033a5",
        "Datetime": "19-07-2025T12:34:55",
        "Location": "IISc Bangalore",
        "From": "userone.amd@gmail.com",
        "Attendees": [
            {
                "email": "usertwo.amd@gmail.com"
            },
            {
                "email": "userthree.amd@gmail.com"
            }
        ],
        "Subject": "queen visiting company",
        "EmailContent": "Hey guys the queen woould like to meet us on Tuesaday at 4 pm"
    }

In [73]:
# Your existing code before calling the function:
creator = input_dict["From"]
try:
    reference_datetime = datetime.strptime(input_dict["Datetime"], "%d-%m-%YT%H:%M:%S")
except ValueError:
    reference_datetime = datetime.fromisoformat(input_dict["Datetime"].replace('T', ' '))

attendees = []
for attendee in input_dict["Attendees"]:
    attendees.append(attendee["email"])
attendees.append(input_dict["From"])

# Replace your entire agent creation and calling code with this:
result = await process_meeting_request(input_dict, agent_model)
print(result)

RUNNING AGENT...
 USER PROMPT: MEETING REQUEST TO PARSE:
Subject: queen visiting company
Email Content: Hey guys the queen woould like to meet us on Tuesaday at 4 pm

Please parse this meeting request and provide the start time, end time, and summary.
🔧 TOOL ACCESSED: YES
🔧 TOOL INPUT - Reference datetime: 2025-07-19 12:34:55
🔧 TOOL INPUT - Target day: tuesday
🔧 TOOL OUTPUT: {'START_TIME': '2025-07-22 09:00:00', 'END_TIME': '2025-07-22 10:00:00'}
🔧 TOOL CALCULATION: Saturday 2025-07-19 + 3 days = Tuesday 2025-07-22
AGENT RESULT: The meeting request is for the queen to visit the company on Tuesday at 4 pm. To parse this request, we first need to find the date of the next Tuesday after the current date.

We use the calculate_meeting_times function to find the date of the next Tuesday. The function returns a dictionary with the start and end times for the meeting.

The start time returned by the function is "2025-07-22 09:00:00". However, the meeting is scheduled for 4 pm, so we need to a

/tmp/ipykernel_166910/2663308455.py:89: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  print(f"AGENT RESULT: {result.data}")


In [66]:
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
import os

# Set environment variables
os.environ["BASE_URL"] = "http://localhost:8000/v1"
os.environ["OPENAI_API_KEY"] = "abc-123"

# Create OpenAI-compatible provider
provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

# Reference the model using the name from --served-model-name
agent_model = OpenAIModel("llama3-70b", provider=provider)


In [70]:
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from typing import Literal

class MeetingSummary(BaseModel):
    START_TIME: str = Field(description="Meeting start time in YYYY-MM-DD HH:MM:SS format")
    END_TIME: str = Field(description="Meeting end time in YYYY-MM-DD HH:MM:SS format")
    SUMMARY: str = Field(description="Brief meeting summary")
    PRIORITY: Literal["1", "2", "3", "4"] = Field(description="Priority level 1-4")


In [71]:
class Event:
    def __init__(self, creator: str,start_time: datetime, end_time: datetime,
                 summary: str, attendees: list[str], priority=None):
        
        # Fields initialized at the creation of the event
        self.creator = creator
        self.start_time = start_time
        self.end_time = end_time
        self.summary = summary
        self.attendees = attendees

        # Fields initialized later in the pipeline, for now set to default values
        curr_time = datetime.now()
        if priority:
            self.priority = priority
        else:
            self.priority: float = 0.0
        self.window_start_time: datetime = datetime.combine(curr_time.date(), time(9, 0)) 
        self.window_end_time: datetime = datetime.combine(curr_time.date(), time(17, 0))
        self.final_start_time: datetime = curr_time
        self.final_end_time: datetime = curr_time


In [78]:
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
import os
from pydantic_ai import Agent
import asyncio
from pydantic_ai.mcp import MCPServerStdio
from datetime import datetime, timedelta, time
from typing import Dict
from datetime import datetime
from pydantic_ai import Tool  
 

# Set environment variables
os.environ["BASE_URL"] = "http://localhost:8000/v1"
os.environ["OPENAI_API_KEY"] = "abc-123"

# Create OpenAI-compatible provider
provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

# Reference the model using the name from --served-model-name
agent_model = OpenAIModel("llama3-70b", provider=provider)

agent = Agent(
    model=agent_model
)

async def run_async(prompt: str) -> str:
    async with agent.run_mcp_servers():
        result = await agent.run(prompt)
        return result.output

    
Tool
def calculate_meeting_times(ref_datetime_str: str, target_day: str) -> Dict[str, str]:
    """
    Given a reference starting datetime string, calculates the start and end datetime
    for a meeting on the next occurrence of a given day of the week.
    
    Args:
        ref_datetime_str: The starting datetime as a string in the format
                          "YYYY-MM-DD HH:MM:SS".
        target_day: The desired day of the week (e.g., "monday", "tuesday", etc.).
    
    Returns:
        A dictionary with "START_TIME" and "END_TIME" strings in YYYY-MM-DD HH:MM:SS format.
    
    Raises:
        ValueError: If an invalid target_day or datetime format is provided.
    """
    
    # # DEBUG: Print when tool is accessed
    # print("🔧 TOOL ACCESSED: YES")
    # print(f"🔧 TOOL INPUT - Reference datetime: {ref_datetime_str}")
    # print(f"🔧 TOOL INPUT - Target day: {target_day}")
    
    try:
        ref_datetime = datetime.strptime(ref_datetime_str, "%Y-%m-%d %H:%M:%S")
    except ValueError:
        raise ValueError("Invalid datetime format. Please use 'YYYY-MM-DD HH:MM:SS'.")
    
    days_of_week = {
        "monday": 0, "tuesday": 1, "wednesday": 2, "thursday": 3,
        "friday": 4, "saturday": 5, "sunday": 6
    }
    
    target_day_lower = target_day.lower()
    if target_day_lower not in days_of_week:
        raise ValueError(f"Invalid target_day: '{target_day}'.")
    
    ref_day_num = ref_datetime.weekday()
    target_day_num = days_of_week[target_day_lower]
    days_to_add = (target_day_num - ref_day_num) % 7
    if days_to_add == 0:  # If it's the same day, get next week
        days_to_add = 7
    
    meeting_date = (ref_datetime + timedelta(days=days_to_add)).date()
    
    # Return with default 9 AM start, 1 hour duration
    start_time_str = f"{meeting_date} 09:00:00"
    end_time_str = f"{meeting_date} 10:00:00"
    
    result = {
        "START_TIME": start_time_str,
        "END_TIME": end_time_str
    }
    
    # # DEBUG: Print what tool returns
    # print(f"🔧 TOOL OUTPUT: {result}")
    # print(f"🔧 TOOL CALCULATION: {ref_datetime.strftime('%A %Y-%m-%d')} + {days_to_add} days = {meeting_date.strftime('%A %Y-%m-%d')}")
    
    return result

async def process_meeting_request(input_dict, agent_model):
    """Process meeting request and return parsed meeting details"""
    
    
    # Parse reference datetime
    try:
        reference_datetime = datetime.strptime(input_dict["Datetime"], "%d-%m-%YT%H:%M:%S")
    except ValueError:
        reference_datetime = datetime.fromisoformat(input_dict["Datetime"].replace('T', ' '))
    
    # Parse attendees
    attendees = []
    for attendee in input_dict["Attendees"]:
        attendees.append(attendee["email"])
    attendees.append(input_dict["From"])
    
    # Create agent with the tool
    agent = Agent(
        model=agent_model,
        output_type=MeetingSummary,
        tools=[calculate_meeting_times],  # Pass the tool here
        system_prompt = f"""You are an expert at parsing meeting requests and calculating dates relative to a reference point. 
        It is of vital importance that you perform the task at hand and only output the FINAL OUTPUT indicated below in the specified final output format. 

REFERENCE INFORMATION:
- Current datetime: {reference_datetime.strftime('%Y-%m-%d %H:%M:%S')}
- Current day: {reference_datetime.strftime('%A')}

CRITICAL TOOL USAGE RULES:
1. When the user mentions a weekday by NAME (Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday):
   - YOU MUST use the calculate_meeting_times(ref_datetime_str, target_day) tool
   - The tool will return the CORRECT DATE - you MUST use this exact date
   - NEVER calculate the date yourself when a weekday name is mentioned
   - Only adjust the TIME portion based on user input (e.g., "4 pm" = 16:00:00)

2. DO NOT use the tool for: "tomorrow", "today", "next week", specific dates like "17th", "January 15th"

STEP-BY-STEP PROCESS FOR WEEKDAY NAMES:
1. Call the tool with the weekday name
2. Extract the DATE from tool's START_TIME (ignore the 09:00:00 time part)
3. Parse the user's requested time (e.g., "4 pm" = 16:00:00)
4. Combine: tool's DATE + user's TIME = final START_TIME
5. Add 1 hour for END_TIME (unless duration specified)

EXAMPLE:
- Tool returns: START_TIME: "2025-07-22 09:00:00" 
- User wants: "Tuesday at 4 pm"
- Your output: START_TIME: "2025-07-22 16:00:00", END_TIME: "2025-07-22 17:00:00"

DATE CALCULATION LOGIC FOR NON-WEEKDAYS:
- "today" = reference date
- "tomorrow" = reference date + 1 day  
- "next [day]" = first occurrence of that day after reference date
- Specific dates: Calculate directly (e.g., "17th" = 17th of current/next month)

TIME PARSING:
- Extract times: "3 pm" -> 15:00, "9:30 AM" -> 09:30
- Default time if none: 09:00
- Default duration: 1 hour
- Time windows: "morning" = 09:00, "afternoon" = 14:00, "evening" = 18:00

PRIORITY LEVELS:
1 = FIXED (unchangeable events like appointments, deadlines, client meetings)
2 = URGENT (time-sensitive but can be slightly adjusted if needed)
3 = IMPORTANT (significant events but with some scheduling flexibility)
4 = FLEXIBLE (can be easily rescheduled or moved)

VERIFICATION STEP:
Before providing your final answer, verify that if you used the tool, your START_TIME uses the same date as the tool's output if it was accessed


FINAL OUTPUT FORMAT:
START_TIME: YYYY-MM-DD HH:MM:SS
END_TIME: YYYY-MM-DD HH:MM:SS
SUMMARY: [Brief meeting summary]
PRIORITY: [1/2/3/4]

"""
    )
    
    # Create user prompt
    user_prompt = f"""MEETING REQUEST TO PARSE:
Subject: {input_dict["Subject"]}
Email Content: {input_dict["EmailContent"]}

Please parse this meeting request and provide the start time, end time, summary and priority. Only output the START_TIME, END_TIME, SUMMARY, PRIORITY as stated in the system prompt. """
    
    # Run the agent
    result = await agent.run(user_prompt)
    return result.output

# write code to convert prioirity to a number

async def main():
    summary = await process_meeting_request(input_dict, agent_model)
    print(f"START_TIME: {summary.START_TIME}")
    print(f"END_TIME: {summary.END_TIME}")
    print(f"SUMMARY: {summary.SUMMARY}")
    print(f"PRIORITY: {summary.PRIORITY}")
    return summary

# For Jupyter notebook - enable nested asyncio
import nest_asyncio
nest_asyncio.apply()

# Run the main function
summary = await main()

# creator, summary.START_TIME, summary.END_TIME , summary.SUMMARY , attendees, summary.PRIORITY
print(summary)
def for_nagasai(creator, start_time_str, end_time_str, summary_text, attendees, priority_str):
    # Convert string datetimes to datetime objects
    start_time = datetime.strptime(start_time_str, "%Y-%m-%d %H:%M:%S")
    end_time = datetime.strptime(end_time_str, "%Y-%m-%d %H:%M:%S")
    
    event = Event(
        creator=creator,
        start_time=start_time,
        end_time=end_time,
        summary=summary_text,
        attendees=attendees,
        priority=float(priority_str) + 0.5
    )
    return event

# Call it with the actual values
event = for_nagasai(creator, summary.START_TIME, summary.END_TIME, summary.SUMMARY, attendees, summary.PRIORITY)


START_TIME: 2025-07-23 16:00:00
END_TIME: 2025-07-23 17:00:00
SUMMARY: Meeting with the queen
PRIORITY: 1
START_TIME='2025-07-23 16:00:00' END_TIME='2025-07-23 17:00:00' SUMMARY='Meeting with the queen' PRIORITY='1'


In [111]:
print(f"START_TIME: {summary.START_TIME}")
print(f"END_TIME: {summary.END_TIME}")
print(f"SUMMARY: {summary.SUMMARY}")
print(f"PRIORITY: {summary.PRIORITY}")

START_TIME: 2025-07-22 16:00:00
END_TIME: 2025-07-22 17:00:00
SUMMARY: Meeting with the queen
PRIORITY: 1


In [64]:
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
import os
from pydantic_ai import Agent
import asyncio
from pydantic_ai.mcp import MCPServerStdio
from datetime import datetime, timedelta, time
from typing import Dict
from datetime import datetime
from pydantic_ai import Tool  
 

# Set environment variables
os.environ["BASE_URL"] = "http://localhost:8000/v1"
os.environ["OPENAI_API_KEY"] = "abc-123"

# Create OpenAI-compatible provider
provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

# Reference the model using the name from --served-model-name
agent_model = OpenAIModel("llama3-8b", provider=provider)

agent = Agent(
    model=agent_model
)

async def run_async(prompt: str) -> str:
    async with agent.run_mcp_servers():
        result = await agent.run(prompt)
        return result.output

    
@Tool
def get_date(ref_datetime_str: str, target_day: str) -> Dict[str, str]:
    """
    Given a reference starting datetime string, calculates the start and end datetime
    for a meeting on the next occurrence of a given day of the week.
    
    Args:
        ref_datetime_str: The starting datetime as a string in the format
                          "YYYY-MM-DD ".
        target_day: The desired day of the week (e.g., "monday", "tuesday", etc.).
    
    Returns:
        A dictionary with "START_DAY" and "END_TIME" strings in YYYY-MM-DDformat.
    
    Raises:
        ValueError: If an invalid target_day or datetime format is provided.
    """
    
    # DEBUG: Print when tool is accessed
    print("🔧 TOOL ACCESSED: YES")
    print(f"🔧 TOOL INPUT - Reference datetime: {ref_datetime_str}")
    print(f"🔧 TOOL INPUT - Target day: {target_day}")

    
    ref_datetime_str = ref_datetime_str[:11].strip()
    try:
        ref_datetime = datetime.strptime(ref_datetime_str, "%Y-%m-%d")
    except ValueError:
        raise ValueError(f"{ref_datetime_str} Invalid datetime format. Please use 'YYYY-MM-DD'.")
    
    days_of_week = {
        "monday": 0, "tuesday": 1, "wednesday": 2, "thursday": 3,
        "friday": 4, "saturday": 5, "sunday": 6
    }
    
    target_day_lower = target_day.lower()
    if target_day_lower not in days_of_week:
        raise ValueError(f"Invalid target_day: '{target_day}'.")
    
    ref_day_num = ref_datetime.weekday()
    target_day_num = days_of_week[target_day_lower]
    days_to_add = (target_day_num - ref_day_num) % 7
    if days_to_add == 0:  # If it's the same day, get next week
        days_to_add = 7
    
    meeting_date = (ref_datetime + timedelta(days=days_to_add)).date()
    
    # Return with default 9 AM start, 1 hour duration
    start_time_str = f"{meeting_date}"

    result = {
        "START_DATE": start_time_str,
    }
    
    # DEBUG: Print what tool returns
    print(f"🔧 TOOL OUTPUT: {result}")
    print(f"🔧 TOOL CALCULATION: {ref_datetime.strftime('%A %Y-%m-%d')} + {days_to_add} days = {meeting_date.strftime('%A %Y-%m-%d')}")
    
    return result

async def process_meeting_request(input_dict, agent_model):
    """Process meeting request and return parsed meeting details"""
    
    # Parse creator and attendees
    creator = input_dict["From"]
    
    # Parse reference datetime
    try:
        reference_datetime = datetime.strptime(input_dict["Datetime"], "%d-%m-%YT%H:%M:%S")
    except ValueError:
        reference_datetime = datetime.fromisoformat(input_dict["Datetime"].replace('T', ' '))
    
    # Parse attendees
    attendees = []
    for attendee in input_dict["Attendees"]:
        attendees.append(attendee["email"])
    attendees.append(input_dict["From"])
    
    # Create agent with the tool
    agent = Agent(
        model=agent_model,
        output_type=MeetingSummary,
        tools=[get_date],  # Pass the tool here
        system_prompt = f"""Take a deep breath you are an expert at parsing meeting requests and calculating dates relative to a reference point.

REFERENCE INFORMATION:
- Current datetime: {reference_datetime.strftime('%Y-%m-%d %H:%M:%S')}
- Current day: {reference_datetime.strftime('%A')}



EXAMPLE:
- Tool returns: START_DAY: "2025-07-22" 
- User wants: "Tuesday"

1. Search the text for occurences of a specific date. If found, return it as START_DATE
2. If there is a day of the week mentioned (eg. Monday, Tuesday etc). Call get_date(ref_datetime_str, weekday) to calculate the specific date of this day.
3. If terms like Today, Tomorrow are used. Do not use the tool, instead, use "today" as the reference date, "tomorrow" as reference date + 1 and so on

In all cases, return a date as START_DATE. Only call get_date() once and never call it again.

VERIFICATION STEP:
Before providing your final answer, verify that if you used the tool, your START_DATE uses the same date as the tool's output if it was accessed
Stop immediately and return the final output once you get  the output from the tool


FINAL OUTPUT FORMAT:
START_DATE: YYYY-MM-DD

Terminate this chat upon sending this answer
"""
    )
    
    # Create user prompt
    user_prompt = f"""MEETING REQUEST TO PARSE:
Subject: {input_dict["Subject"]}
Email Content: {input_dict["EmailContent"]}

Please parse this meeting request and provide the Start date and end date"""
    
    # Run the agent
    result = await agent.run(user_prompt)
    return result.output

input_dict = {
        "Request_id": "6118b54f-907b-a4451-8d48-dd13d76033a5",
        "Datetime": "19-07-2025T12:34:55",
        "Location": "IISc Bangalore",
        "From": "userone.amd@gmail.com",
        "Attendees": [
            {
                "email": "usertwo.amd@gmail.com"
            },
            {
                "email": "userthree.amd@gmail.com"
            }
        ],
        "Subject": "queen visiting company",
        "EmailContent": "Hey guys the queen woould like to meet us on Tuesday at 4 pm"
    }

creator = input_dict["From"]
try:
    reference_datetime = datetime.strptime(input_dict["Datetime"], "%d-%m-%YT%H:%M:%S")
except ValueError:
    reference_datetime = datetime.fromisoformat(input_dict["Datetime"].replace('T', ' '))

attendees = []
for attendee in input_dict["Attendees"]:
    attendees.append(attendee["email"])
attendees.append(input_dict["From"])

# write code to convert prioirity to a number

async def main():
    summary = await process_meeting_request(input_dict, agent_model)
    print(f"START_DATE: {summary.START_DATE}")
    return summary

# For Jupyter notebook - enable nested asyncio
import nest_asyncio
nest_asyncio.apply()

# Run the main function
summary = await main()

# creator, summary.START_TIME, summary.END_TIME , summary.SUMMARY , attendees, summary.PRIORITY
print(summary)
# def for_nagasai(creator, start_time_str, end_time_str, summary_text, attendees, priority_str):
#     # Convert string datetimes to datetime objects
#     start_time = datetime.strptime(start_time_str, "%Y-%m-%d %H:%M:%S")
#     end_time = datetime.strptime(end_time_str, "%Y-%m-%d %H:%M:%S")
    
#     event = Event(
#         creator=creator,
#         start_time=start_time,
#         end_time=end_time,
#         summary=summary_text,
#         attendees=attendees,
#         priority=float(priority_str) + 0.5
#     )
#     return event

# Call it with the actual values
#event = for_nagasai(creator, summary.START_TIME, summary.END_TIME, summary.SUMMARY, attendees, summary.PRIORITY)


🔧 TOOL ACCESSED: YES
🔧 TOOL INPUT - Reference datetime: 
🔧 TOOL INPUT - Target day: 
🔧 TOOL ACCESSED: YES
🔧 TOOL INPUT - Reference datetime: 
🔧 TOOL INPUT - Target day: 


ValueError:  Invalid datetime format. Please use 'YYYY-MM-DD'.

In [ ]:
"""
CRITICAL TOOL USAGE RULES:
1. When the user mentions a weekday by NAME (Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday):
   - YOU MUST use the get_date(ref_datetime_str, target_day) tool
   - The tool will return the CORRECT DATE - you MUST use this exact date
   - NEVER calculate the date yourself when a weekday name is mentioned

2. DO NOT use the tool for: "tomorrow", "today", "next week", specific dates like "17th", "January 15th"

STEP-BY-STEP PROCESS FOR WEEKDAY NAMES:
1. Call the tool with the weekday name
2. Extract the DATE from tool's reference date time
3. Parse the user's requested time 
4. Combine: tool's DATE 

"""

In [ ]:
#trying to modify the final prompt from here on out